In [1]:
# Connection Strings - json

# Connection Strings - SQL database (where we are writing to)

# Translate json file into objects

# Fstrings to write dynamic SQL code into tables

# Close connection strings

In [2]:
# Import Depencies

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import numpy as np
import requests
from pprint import pprint
import time
import json
from requests.structures import CaseInsensitiveDict

from apikey import apikey

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [3]:
#Section for user input

lat = ''
lon = ''

default = [0, 0]
germany = [51.1657, 10.4515]
london = [51.507351, -0.127758]
brisbane = [-27.469770, 153.025131]

location_search = [123456789,123456789] #user_defined or written to pre-determined

default_city_list =  {'Germany' : germany , 'London' : london, 'Brisbane' : brisbane}


print('Available list of cities\n')
for k, v in default_city_list.items():
    print(k)



Available list of cities

Germany
London
Brisbane


In [4]:
user_coordinates = input("Would you like pick from a list of available cities? yes or no")

if user_coordinates.lower() == 'y' or user_coordinates.lower() == 'yes':

    match = input("Please pick a city from the list.\n")

    for k, v in default_city_list.items():
        if match.lower() == k.lower():
            location_search = v

elif user_coordinates.lower() == 'n' or user_coordinates.lower() == 'no':
    user_lat = input("Please enter the latitude. ")
    user_lon = input("Please enter the longitude. ")
    location_search = [float(user_lat), float(user_lon)]

else:
    raise Exception("Invalid Input. Must specify Yes (y) or No (n)")

if location_search == [123456789, 123456789]:
    raise Exception("Invalid location. Use a name that is in the list, or choose No (n) your own coordinates.")
    
print(f'Using coordinates {location_search}\n')


Using coordinates [25.7617, -80.1918]



In [5]:



#get latitude, longitude, and radius for starting point - New York City is [40.712776 N, -74.005974 W], Germany is [51.1657 N, 10.4515 E]  Germany country code is "de"
#London is [51.507351, -0.127758] Brisbane is [-27.469770, 153.025131]
#I can find London, but US cities are not coming up





#THESE VALUES ARE JUST FOR TESTING. DELETE 
#location_search = germany
#lat = "-27.469770"
#lon = "153.025131"

#User inputs should be placed for these too - Just ask if the default values are OK.
radius_meters = 5000
country_code = ''
limit = 100
offset = 0 #default is zero

location_filter = f'circle:{location_search[1]},{location_search[0]},{radius_meters}'

if country_code != "":
    country_code = country_code + f"|countrycode:{country_code}"

headers = CaseInsensitiveDict()
headers["Accept"] = "application/json"

#resp = requests.get(url, headers=headers)

#print(resp.status_code)

In [7]:
# List of categories
# catering, commercial.food_and_drinks, accomodations, entertainment, leisure, parking, wheelchair
category_string = ""

activity = True
commercial = True
commercial_catering = False
commercial_supermarket = True
accomodations = False
entertainment = True
leisure = False
parking = False
wheelchair = False


In [8]:
# dictionary of each category that will be captured
# we also may want to include user input for this too. Loop through these, or other questions. Set value to True. Default value is false. Must have at least one of these active in order
# to run the script. If not, throw an error
# We either need to specify that the list needs to be under 500, because that is the max number, or create 
category_dict = {'activity' : activity, 'commercial' : commercial, 'commerical.catering' : commercial_catering, 'accomodation' : accomodations, \
                        'entertainment' : entertainment, 'leisure': leisure, 'parking' : parking, 'wheelchair.yes' : wheelchair}

string_of_categories = ""

for k,v in category_dict.items():
    if v:
        string_of_categories = string_of_categories + k + ','

#remove last comma
string_of_categories = string_of_categories[:-1]

# conditions, organic.only, vegetarian.only, wheelchair.yes
url2 = f"https://api.geoapify.com/v2/places?categories={string_of_categories}&filter={location_filter}&limit={limit}&offset={offset}&apiKey={apikey}"

# atms in radius, 

# contact information for the restaurants, opening/closing hours, web address, phone number

In [9]:
# get API dictionary
resp = requests.get(url2, headers=headers)

print(resp.status_code)

json_info = resp.json()
pprint(json_info)

NameError: name 'headers' is not defined

In [31]:
pd.json_normalize(json_info["features"]).columns.values

array(['type', 'properties.name', 'properties.country',
       'properties.country_code', 'properties.state', 'properties.county',
       'properties.city', 'properties.postcode', 'properties.street',
       'properties.housenumber', 'properties.lon', 'properties.lat',
       'properties.state_code', 'properties.formatted',
       'properties.address_line1', 'properties.address_line2',
       'properties.categories', 'properties.details',
       'properties.datasource.sourcename',
       'properties.datasource.attribution',
       'properties.datasource.license', 'properties.datasource.url',
       'properties.datasource.raw.name',
       'properties.datasource.raw.level',
       'properties.datasource.raw.phone',
       'properties.datasource.raw.sport',
       'properties.datasource.raw.osm_id',
       'properties.datasource.raw.tourism',
       'properties.datasource.raw.website',
       'properties.datasource.raw.building',
       'properties.datasource.raw.end_date',
       'prope

In [32]:
features_pd = pd.json_normalize(json_info["features"])

In [74]:
properties_df = features_pd[["properties.address_line1", "properties.address_line2", "properties.place_id"]]
properties_df

,properties.address_line1,properties.address_line2,properties.place_id
0,Georgia Aquarium,"225 Baker Street Northwest, Atlanta, GA 30313,...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,Carter Presidential Library,"441 John Lewis Freedom Parkway Northeast, Atla...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,Spelman College Museum of Fine Art,"350 Greensferry Avenue Southwest, Atlanta, GA ...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,Atlanta Contemporary Art Center,"535 Means Street Northwest, Atlanta, GA 30318,...",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,High Museum of Art,"1280 Peachtree Street Northeast, Atlanta, GA 3...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...,...
95,The Tabernacle,"152 Luckie Street Northwest, Atlanta, GA 30303...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,Museum of Design Atlanta (MODA),"16th Street Northeast, Atlanta, GA 30309, Unit...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,Trap Music Museum,"630 Travis Street Northwest, Atlanta, GA 30318...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,The Spotted Trotter,"229 Moreland Avenue Southeast, Atlanta, GA 303...",513e22b277601655c0596ed93cbca0df4040f00102f901...


In [75]:
# Rename columns
properties_df = properties_df.rename(columns = {"properties.address_line1": "Property", "properties.address_line2": "Address2", "properties.place_id": "Place_ID"})
properties_df

,Property,Address2,Place_ID
0,Georgia Aquarium,"225 Baker Street Northwest, Atlanta, GA 30313,...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,Carter Presidential Library,"441 John Lewis Freedom Parkway Northeast, Atla...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,Spelman College Museum of Fine Art,"350 Greensferry Avenue Southwest, Atlanta, GA ...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,Atlanta Contemporary Art Center,"535 Means Street Northwest, Atlanta, GA 30318,...",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,High Museum of Art,"1280 Peachtree Street Northeast, Atlanta, GA 3...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...,...
95,The Tabernacle,"152 Luckie Street Northwest, Atlanta, GA 30303...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,Museum of Design Atlanta (MODA),"16th Street Northeast, Atlanta, GA 30309, Unit...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,Trap Music Museum,"630 Travis Street Northwest, Atlanta, GA 30318...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,The Spotted Trotter,"229 Moreland Avenue Southeast, Atlanta, GA 303...",513e22b277601655c0596ed93cbca0df4040f00102f901...


In [76]:
# Create separate columns for the address fields
properties_df[['Address', 'City', 'State_Zip', 'Country']] = properties_df.Address2.str.split(",", expand = True)
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Georgia Aquarium,"225 Baker Street Northwest, Atlanta, GA 30313,...",5114eebb6d471955c059ba64831cb3e14040f00102f901...,225 Baker Street Northwest,Atlanta,GA 30313,United States of America
1,Carter Presidential Library,"441 John Lewis Freedom Parkway Northeast, Atla...",51218fb053d81655c05947c83a8e2ce24040f00102f901...,441 John Lewis Freedom Parkway Northeast,Atlanta,GA 30307,United States of America
2,Spelman College Museum of Fine Art,"350 Greensferry Avenue Southwest, Atlanta, GA ...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...,350 Greensferry Avenue Southwest,Atlanta,GA 30314,United States of America
3,Atlanta Contemporary Art Center,"535 Means Street Northwest, Atlanta, GA 30318,...",511ade077bee1955c05975b415d3f1e24040f00103f901...,535 Means Street Northwest,Atlanta,GA 30318,United States of America
4,High Museum of Art,"1280 Peachtree Street Northeast, Atlanta, GA 3...",511e9d688ab31855c059fb9730ff15e54040f00102f901...,1280 Peachtree Street Northeast,Atlanta,GA 30309,United States of America
...,...,...,...,...,...,...,...
95,The Tabernacle,"152 Luckie Street Northwest, Atlanta, GA 30303...",51d466fec60d1955c059dd74045a1de14040f00102f901...,152 Luckie Street Northwest,Atlanta,GA 30303,United States of America
96,Museum of Design Atlanta (MODA),"16th Street Northeast, Atlanta, GA 30309, Unit...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...,16th Street Northeast,Atlanta,GA 30309,United States of America
97,Trap Music Museum,"630 Travis Street Northwest, Atlanta, GA 30318...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...,630 Travis Street Northwest,Atlanta,GA 30318,United States of America
98,The Spotted Trotter,"229 Moreland Avenue Southeast, Atlanta, GA 303...",513e22b277601655c0596ed93cbca0df4040f00102f901...,229 Moreland Avenue Southeast,Atlanta,GA 30316,United States of America


In [77]:
properties_df = properties_df.dropna()
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Georgia Aquarium,"225 Baker Street Northwest, Atlanta, GA 30313,...",5114eebb6d471955c059ba64831cb3e14040f00102f901...,225 Baker Street Northwest,Atlanta,GA 30313,United States of America
1,Carter Presidential Library,"441 John Lewis Freedom Parkway Northeast, Atla...",51218fb053d81655c05947c83a8e2ce24040f00102f901...,441 John Lewis Freedom Parkway Northeast,Atlanta,GA 30307,United States of America
2,Spelman College Museum of Fine Art,"350 Greensferry Avenue Southwest, Atlanta, GA ...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...,350 Greensferry Avenue Southwest,Atlanta,GA 30314,United States of America
3,Atlanta Contemporary Art Center,"535 Means Street Northwest, Atlanta, GA 30318,...",511ade077bee1955c05975b415d3f1e24040f00103f901...,535 Means Street Northwest,Atlanta,GA 30318,United States of America
4,High Museum of Art,"1280 Peachtree Street Northeast, Atlanta, GA 3...",511e9d688ab31855c059fb9730ff15e54040f00102f901...,1280 Peachtree Street Northeast,Atlanta,GA 30309,United States of America
...,...,...,...,...,...,...,...
94,King Birth Home,"501 Auburn Avenue Northeast, Atlanta, GA 30312...",515698d627c11755c0595912f449abe04040f00102f901...,501 Auburn Avenue Northeast,Atlanta,GA 30312,United States of America
95,The Tabernacle,"152 Luckie Street Northwest, Atlanta, GA 30303...",51d466fec60d1955c059dd74045a1de14040f00102f901...,152 Luckie Street Northwest,Atlanta,GA 30303,United States of America
96,Museum of Design Atlanta (MODA),"16th Street Northeast, Atlanta, GA 30309, Unit...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...,16th Street Northeast,Atlanta,GA 30309,United States of America
97,Trap Music Museum,"630 Travis Street Northwest, Atlanta, GA 30318...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...,630 Travis Street Northwest,Atlanta,GA 30318,United States of America


In [78]:
properties_df["State_Zip"] = properties_df['State_Zip'].str.strip()
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Georgia Aquarium,"225 Baker Street Northwest, Atlanta, GA 30313,...",5114eebb6d471955c059ba64831cb3e14040f00102f901...,225 Baker Street Northwest,Atlanta,GA 30313,United States of America
1,Carter Presidential Library,"441 John Lewis Freedom Parkway Northeast, Atla...",51218fb053d81655c05947c83a8e2ce24040f00102f901...,441 John Lewis Freedom Parkway Northeast,Atlanta,GA 30307,United States of America
2,Spelman College Museum of Fine Art,"350 Greensferry Avenue Southwest, Atlanta, GA ...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...,350 Greensferry Avenue Southwest,Atlanta,GA 30314,United States of America
3,Atlanta Contemporary Art Center,"535 Means Street Northwest, Atlanta, GA 30318,...",511ade077bee1955c05975b415d3f1e24040f00103f901...,535 Means Street Northwest,Atlanta,GA 30318,United States of America
4,High Museum of Art,"1280 Peachtree Street Northeast, Atlanta, GA 3...",511e9d688ab31855c059fb9730ff15e54040f00102f901...,1280 Peachtree Street Northeast,Atlanta,GA 30309,United States of America
...,...,...,...,...,...,...,...
94,King Birth Home,"501 Auburn Avenue Northeast, Atlanta, GA 30312...",515698d627c11755c0595912f449abe04040f00102f901...,501 Auburn Avenue Northeast,Atlanta,GA 30312,United States of America
95,The Tabernacle,"152 Luckie Street Northwest, Atlanta, GA 30303...",51d466fec60d1955c059dd74045a1de14040f00102f901...,152 Luckie Street Northwest,Atlanta,GA 30303,United States of America
96,Museum of Design Atlanta (MODA),"16th Street Northeast, Atlanta, GA 30309, Unit...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...,16th Street Northeast,Atlanta,GA 30309,United States of America
97,Trap Music Museum,"630 Travis Street Northwest, Atlanta, GA 30318...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...,630 Travis Street Northwest,Atlanta,GA 30318,United States of America


In [80]:
properties_df.State_Zip.str.split(" ", expand = True)

,0,1
0,GA,30313
1,GA,30307
2,GA,30314
3,GA,30318
4,GA,30309
...,...,...
94,GA,30312
95,GA,30303
96,GA,30309
97,GA,30318


In [81]:
properties_df[['State', 'Zip']] = properties_df.State_Zip.str.split(" ", expand = True)
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country,State,Zip
0,Georgia Aquarium,"225 Baker Street Northwest, Atlanta, GA 30313,...",5114eebb6d471955c059ba64831cb3e14040f00102f901...,225 Baker Street Northwest,Atlanta,GA 30313,United States of America,GA,30313
1,Carter Presidential Library,"441 John Lewis Freedom Parkway Northeast, Atla...",51218fb053d81655c05947c83a8e2ce24040f00102f901...,441 John Lewis Freedom Parkway Northeast,Atlanta,GA 30307,United States of America,GA,30307
2,Spelman College Museum of Fine Art,"350 Greensferry Avenue Southwest, Atlanta, GA ...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...,350 Greensferry Avenue Southwest,Atlanta,GA 30314,United States of America,GA,30314
3,Atlanta Contemporary Art Center,"535 Means Street Northwest, Atlanta, GA 30318,...",511ade077bee1955c05975b415d3f1e24040f00103f901...,535 Means Street Northwest,Atlanta,GA 30318,United States of America,GA,30318
4,High Museum of Art,"1280 Peachtree Street Northeast, Atlanta, GA 3...",511e9d688ab31855c059fb9730ff15e54040f00102f901...,1280 Peachtree Street Northeast,Atlanta,GA 30309,United States of America,GA,30309
...,...,...,...,...,...,...,...,...,...
94,King Birth Home,"501 Auburn Avenue Northeast, Atlanta, GA 30312...",515698d627c11755c0595912f449abe04040f00102f901...,501 Auburn Avenue Northeast,Atlanta,GA 30312,United States of America,GA,30312
95,The Tabernacle,"152 Luckie Street Northwest, Atlanta, GA 30303...",51d466fec60d1955c059dd74045a1de14040f00102f901...,152 Luckie Street Northwest,Atlanta,GA 30303,United States of America,GA,30303
96,Museum of Design Atlanta (MODA),"16th Street Northeast, Atlanta, GA 30309, Unit...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...,16th Street Northeast,Atlanta,GA 30309,United States of America,GA,30309
97,Trap Music Museum,"630 Travis Street Northwest, Atlanta, GA 30318...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...,630 Travis Street Northwest,Atlanta,GA 30318,United States of America,GA,30318


In [90]:
properties_df = properties_df.drop('Address2', axis=1)
properties_df

,Property,Place_ID,Address,City,State_Zip,Country,State,Zip
0,Georgia Aquarium,5114eebb6d471955c059ba64831cb3e14040f00102f901...,225 Baker Street Northwest,Atlanta,GA 30313,United States of America,GA,30313
1,Carter Presidential Library,51218fb053d81655c05947c83a8e2ce24040f00102f901...,441 John Lewis Freedom Parkway Northeast,Atlanta,GA 30307,United States of America,GA,30307
2,Spelman College Museum of Fine Art,5186dc3b2e651a55c059d19081fd85df4040f00102f901...,350 Greensferry Avenue Southwest,Atlanta,GA 30314,United States of America,GA,30314
3,Atlanta Contemporary Art Center,511ade077bee1955c05975b415d3f1e24040f00103f901...,535 Means Street Northwest,Atlanta,GA 30318,United States of America,GA,30318
4,High Museum of Art,511e9d688ab31855c059fb9730ff15e54040f00102f901...,1280 Peachtree Street Northeast,Atlanta,GA 30309,United States of America,GA,30309
...,...,...,...,...,...,...,...,...
94,King Birth Home,515698d627c11755c0595912f449abe04040f00102f901...,501 Auburn Avenue Northeast,Atlanta,GA 30312,United States of America,GA,30312
95,The Tabernacle,51d466fec60d1955c059dd74045a1de14040f00102f901...,152 Luckie Street Northwest,Atlanta,GA 30303,United States of America,GA,30303
96,Museum of Design Atlanta (MODA),516f4e3da59c1855c0596f2cbf1236e54040f00102f901...,16th Street Northeast,Atlanta,GA 30309,United States of America,GA,30309
97,Trap Music Museum,51e3a7cc00281a55c059e5973d76cae24040f00102f901...,630 Travis Street Northwest,Atlanta,GA 30318,United States of America,GA,30318


In [91]:
properties_df = properties_df.drop('State_Zip', axis=1)
properties_df

,Property,Place_ID,Address,City,Country,State,Zip
0,Georgia Aquarium,5114eebb6d471955c059ba64831cb3e14040f00102f901...,225 Baker Street Northwest,Atlanta,United States of America,GA,30313
1,Carter Presidential Library,51218fb053d81655c05947c83a8e2ce24040f00102f901...,441 John Lewis Freedom Parkway Northeast,Atlanta,United States of America,GA,30307
2,Spelman College Museum of Fine Art,5186dc3b2e651a55c059d19081fd85df4040f00102f901...,350 Greensferry Avenue Southwest,Atlanta,United States of America,GA,30314
3,Atlanta Contemporary Art Center,511ade077bee1955c05975b415d3f1e24040f00103f901...,535 Means Street Northwest,Atlanta,United States of America,GA,30318
4,High Museum of Art,511e9d688ab31855c059fb9730ff15e54040f00102f901...,1280 Peachtree Street Northeast,Atlanta,United States of America,GA,30309
...,...,...,...,...,...,...,...
94,King Birth Home,515698d627c11755c0595912f449abe04040f00102f901...,501 Auburn Avenue Northeast,Atlanta,United States of America,GA,30312
95,The Tabernacle,51d466fec60d1955c059dd74045a1de14040f00102f901...,152 Luckie Street Northwest,Atlanta,United States of America,GA,30303
96,Museum of Design Atlanta (MODA),516f4e3da59c1855c0596f2cbf1236e54040f00102f901...,16th Street Northeast,Atlanta,United States of America,GA,30309
97,Trap Music Museum,51e3a7cc00281a55c059e5973d76cae24040f00102f901...,630 Travis Street Northwest,Atlanta,United States of America,GA,30318


In [84]:
no_duplicates_df = properties_df.drop_duplicates(subset = "Place_ID", inplace = True)
no_duplicates_df

In [86]:
categories_df = features_pd[["properties.categories", "properties.place_id"]]
categories_df

,properties.categories,properties.place_id
0,"[building, building.sport, building.tourism, e...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,"[building, building.tourism, entertainment, en...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,"[building, building.tourism, entertainment, en...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,"[entertainment, entertainment.museum]",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,"[building, building.tourism, entertainment, en...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...
95,"[building, building.commercial, building.enter...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,"[building, building.tourism, entertainment, en...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,"[building, building.tourism, entertainment, en...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,"[building, building.commercial, commercial, co...",513e22b277601655c0596ed93cbca0df4040f00102f901...


In [88]:
# Rename columns
categories_df = categories_df.rename(columns = {"properties.categories": "Categories", "properties.place_id": "Place_ID"})
categories_df

,Categories,Place_ID
0,"[building, building.sport, building.tourism, e...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,"[building, building.tourism, entertainment, en...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,"[building, building.tourism, entertainment, en...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,"[entertainment, entertainment.museum]",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,"[building, building.tourism, entertainment, en...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...
95,"[building, building.commercial, building.enter...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,"[building, building.tourism, entertainment, en...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,"[building, building.tourism, entertainment, en...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,"[building, building.commercial, commercial, co...",513e22b277601655c0596ed93cbca0df4040f00102f901...


In [89]:
categories_df = categories_df.dropna()
categories_df

,Categories,Place_ID
0,"[building, building.sport, building.tourism, e...",5114eebb6d471955c059ba64831cb3e14040f00102f901...
1,"[building, building.tourism, entertainment, en...",51218fb053d81655c05947c83a8e2ce24040f00102f901...
2,"[building, building.tourism, entertainment, en...",5186dc3b2e651a55c059d19081fd85df4040f00102f901...
3,"[entertainment, entertainment.museum]",511ade077bee1955c05975b415d3f1e24040f00103f901...
4,"[building, building.tourism, entertainment, en...",511e9d688ab31855c059fb9730ff15e54040f00102f901...
...,...,...
95,"[building, building.commercial, building.enter...",51d466fec60d1955c059dd74045a1de14040f00102f901...
96,"[building, building.tourism, entertainment, en...",516f4e3da59c1855c0596f2cbf1236e54040f00102f901...
97,"[building, building.tourism, entertainment, en...",51e3a7cc00281a55c059e5973d76cae24040f00102f901...
98,"[building, building.commercial, commercial, co...",513e22b277601655c0596ed93cbca0df4040f00102f901...
